In [1]:
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import json

import pandas as pd
from tqdm import tqdm
import os

In [9]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# chat_model = ChatOpenAI(api_key=OPENAI_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index('receipe')

In [3]:
# response = client.embeddings.create(
#     input="hello",
#     model="text-embedding-ada-002"
# )

# print(response.data[0].embedding)

[-0.025150660425424576, -0.019543994218111038, -0.027953993529081345, -0.03099534474313259, -0.024740738794207573, 0.027583742514252663, -0.012634835205972195, -0.008449669927358627, -0.017613396048545837, -0.008284378796815872, 0.03250279650092125, 0.004304174333810806, -0.024582060053944588, -0.0006231465958990157, 0.014135675504803658, -0.0016231564804911613, 0.03943179175257683, 0.0021504343021661043, 0.027041587978601456, -0.012489378452301025, -0.021091116592288017, 0.008965377695858479, 0.00839016493409872, -0.003259536111727357, -0.0053917886689305305, -0.00962654035538435, 0.011219944804906845, -0.0015396844828501344, 0.003557059681043029, -0.023286178708076477, 0.006783537566661835, -0.007854622788727283, -0.023907672613859177, -0.008912484161555767, 0.006862877402454615, -0.013778647407889366, 0.009533978067338467, -0.01420179195702076, 0.021990299224853516, -0.01057861652225256, 0.0033884630538523197, -0.014664607122540474, 0.00538187101483345, -0.014862955547869205, -0.031

In [3]:
proc = pd.read_csv('./data/whole_processed.csv')
proc.head()

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,128932,입안에서 톡톡톡,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
3,131871,★현미호두죽,현미호두죽,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내


In [4]:
proc10 = proc.iloc[:100, :]

In [10]:
list_without_recipe_id = proc10.drop(columns=['Recipe_ID']).apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_without_recipe_id

['Recipe_Title: 어묵김말이, Dish_Name: 어묵김말이, Cooking_Method: 튀김, Cooking_Status: 간식, Ingredient_List: 가공식품류, Dish_Type: 디저트, Cooking_Intro: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-, Ingredient_Details: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 60분이내',
 'Recipe_Title: 두부에 꼬리가 달렸어요!!, Dish_Name: 두부새우전, Cooking_Method: 부침, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밑반찬, Cooking_Intro: 꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥, Ingredient_Details: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개, Serving_Size: 3인분, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 입안에서 톡톡톡, Dish_Name: 알밥, Cooking_Method: 굽기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥, Ingredient_Details: [재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간|

In [5]:
# --------------------------------=
# |레시피 임베딩 & vectorDB에 업로드|
# ---------------------------------

# embeddings = []
# for i in tqdm(list_without_recipe_id):
#     response = client.embeddings.create(
#         input=i,
#         model="text-embedding-ada-002"
#     )
#     embeddings.append(response.data[0].embedding)

# embeddings

# ids = proc10['Recipe_ID'].to_list()
# ids

# meta_datas = [{'text': text} for text in list_without_recipe_id]
# meta_datas

# records = [{'id': str(i), 'values': j, 'metadata': k} for i,j,k in zip(ids, embeddings, meta_datas)]
# records

# index.upsert(vectors=records)

In [6]:
# vectorDB에 검색하는 함수
def request_query(query):
    response = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002"
    )

    embedded_query=response.data[0].embedding
    results=index.query(embedded_query, top_k=3, include_metadata=True)

    return results['matches']

In [12]:
user_need = '뜨끈한 해물이 포함된 국물이 있는 요리.'
ingredients = '오징어| 새우| 어묵| 조개| 홍합| 고추가루'

In [13]:
pinecone_result = request_query(f'{user_need}, Ingredient_Details: {ingredients}')
recipes = [i['metadata']['text'] for i in pinecone_result]
recipes

['Recipe_Title: 매콤 오징어 볶음우동, Dish_Name: 오징어볶음우동, Cooking_Method: 볶음, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 면/만두, Cooking_Intro: 오늘은 오징어를 맛있게 볶아봤어요~ 보통 오징어볶음은 소면이랑 같이 많이들 드시잖아요~ 근데 소면대신 우동사리를 넣으면 오통통하게 씹히는 맛이 더 좋답니다~ 보기에는 매워도 막상 먹을때는 많이 맵지 않아서~ 아이들과 함께 드실 수 있어요~, Ingredient_Details: [재료] 오징어 1마리| 양파 1/4개| 대파 1/2쪽| 당근 1/3개| 청고추 1개| 홍고추 1개 [양념] 고추장 1T| 고춧가루 1T| 다진 마늘 1T| 참기름 1T| 설탕 1t| 간장 1t| 청주 1t| 후추 약간| 소금 약간, Serving_Size: 2인분, Difficulty_Level: 중급, Cooking_Time: 30분이내',
 'Recipe_Title: 해물을 가득 넣은 뚝배기 해물떡볶이, Dish_Name: 뚝배기해물떡볶이, Cooking_Method: 볶음, Cooking_Status: 술안주, Ingredient_List: 해물류, Dish_Type: 밥/죽/떡, Cooking_Intro: 해물을 가득가득 넣은 떡볶이를 만들어 보아요~ 보통먹는 자잘한 떡볶이가 아니라 볶아 만드는 떡볶이랍니다~ 일명 안주용 떡볶이죠~ 맛있는 떡볶이 함께 만들어 보아요~, Ingredient_Details: [재료] 떡볶이떡 150g| 당근 1/2개| 파 1뿌리| 양파 1/2개| 양배추 1/4통| 어묵 2장| 오징어| 새우| 조갯살| 홍합살 [양념장] 고춧가루 2T| 고추장 1T| 물엿 1T| 간장 1T| 참기름 1T| 다진 마늘 1/2T| 깨 1/2T| 후추가루 약간, Serving_Size: 2인분, Difficulty_Level: 중급, Cooking_Time: 30분이내',
 'Recipe_Title: 새우볶음밥 고로케, D

In [19]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "당신은 한식 최고의 요리사입니다. 추천 요리 세 가지를 활용하여 레시피를 알려주세요."},
        {"role": "user", "content": f"추천요리1: {recipes[0]}\n 추천요리2: {recipes[1]}\n 추천요리3: {recipes[2]}"}
    ]
)
print(response.choices[0].message.content)

추천요리 1: **매콤 오징어 볶음우동**

**재료:**
- 오징어 1마리
- 양파 1/4개
- 대파 1/2쪽
- 당근 1/3개
- 청고추 1개
- 홍고추 1개
- 우동사리 2개
- [양념] 고추장 1T, 고춧가루 1T, 다진 마늘 1T, 참기름 1T, 설탕 1t, 간장 1t, 청주 1t, 후추 약간, 소금 약간

**조리과정:**
1. 오징어는 깨끗이 씻어 썰고, 양파, 대파, 당근, 청고추, 홍고추는 얇게 채 썬다.
2. 큰 팬에 식용유를 두르고 양파, 대파, 당근을 중불에서 볶다가 오징어를 추가한다.
3. 오징어가 반쯤 익으면 고추장, 고춧가루, 다진 마늘, 간장, 설탕, 청주, 후추를 넣고 잘 섞는다.
4. 오징어와 야채가 잘 섞이고 익으면 우동사리를 넣고 함께 볶는다.
5. 참기름을 넣고 섞은 후 맛을 보아 소금으로 간을 조절한다.
6. 마지막으로 청고추와 홍고추를 넣고 한 번 더 섞어준 뒤 접시에 담는다.

**걸리는 시간:** 30분

**난이도:** 중급

추천요리 2: **해물을 가득 넣은 뚝배기 해물떡볶이**

**재료:**
- 떡볶이떡 150g
- 당근 1/2개
- 파 1뿌리
- 양파 1/2개
- 양배추 1/4통
- 어묵 2장
- 오징어 1마리
- 새우 10마리
- 조갯살 100g
- 홍합살 100g
- [양념장] 고춧가루 2T, 고추장 1T, 물엿 1T, 간장 1T, 참기름 1T, 다진 마늘 1/2T, 깨 1/2T, 후추가루 약간

**조리과정:**
1. 모든 해물은 깨끗이 씻어 준비하고, 떡볶이떡은 찬물에 잠시 담가둔다.
2. 당근, 양파, 양배추를 얇게 채 썬다. 어묵도 먹기 좋은 크기로 자른다.
3. 냄비에 물을 조금 붓고 끓이다가 떡볶이떡을 넣고 살짝 익힌다.
4. 해물과 어묵을 넣고, 준비한 양념장을 넣고 잘 섞는다.
5. 채 썬 야채를 넣고 모두 잘 섞어주며 볶는다.
6. 모든 재료가 익으면 참기름을 두르고 살짝 더 볶아서 마무리한다.
7. 그릇에 담아 깨를 뿌려 완성한다.

**걸리는 시간:** 30분

**난이도:** 중급

In [21]:
print(response.choices[0].message.content)

추천요리 1: **매콤 오징어 볶음우동**

**재료:**
- 오징어 1마리
- 양파 1/4개
- 대파 1/2쪽
- 당근 1/3개
- 청고추 1개
- 홍고추 1개
- 우동사리 2개
- [양념] 고추장 1T, 고춧가루 1T, 다진 마늘 1T, 참기름 1T, 설탕 1t, 간장 1t, 청주 1t, 후추 약간, 소금 약간

**조리과정:**
1. 오징어는 깨끗이 씻어 썰고, 양파, 대파, 당근, 청고추, 홍고추는 얇게 채 썬다.
2. 큰 팬에 식용유를 두르고 양파, 대파, 당근을 중불에서 볶다가 오징어를 추가한다.
3. 오징어가 반쯤 익으면 고추장, 고춧가루, 다진 마늘, 간장, 설탕, 청주, 후추를 넣고 잘 섞는다.
4. 오징어와 야채가 잘 섞이고 익으면 우동사리를 넣고 함께 볶는다.
5. 참기름을 넣고 섞은 후 맛을 보아 소금으로 간을 조절한다.
6. 마지막으로 청고추와 홍고추를 넣고 한 번 더 섞어준 뒤 접시에 담는다.

**걸리는 시간:** 30분

**난이도:** 중급

추천요리 2: **해물을 가득 넣은 뚝배기 해물떡볶이**

**재료:**
- 떡볶이떡 150g
- 당근 1/2개
- 파 1뿌리
- 양파 1/2개
- 양배추 1/4통
- 어묵 2장
- 오징어 1마리
- 새우 10마리
- 조갯살 100g
- 홍합살 100g
- [양념장] 고춧가루 2T, 고추장 1T, 물엿 1T, 간장 1T, 참기름 1T, 다진 마늘 1/2T, 깨 1/2T, 후추가루 약간

**조리과정:**
1. 모든 해물은 깨끗이 씻어 준비하고, 떡볶이떡은 찬물에 잠시 담가둔다.
2. 당근, 양파, 양배추를 얇게 채 썬다. 어묵도 먹기 좋은 크기로 자른다.
3. 냄비에 물을 조금 붓고 끓이다가 떡볶이떡을 넣고 살짝 익힌다.
4. 해물과 어묵을 넣고, 준비한 양념장을 넣고 잘 섞는다.
5. 채 썬 야채를 넣고 모두 잘 섞어주며 볶는다.
6. 모든 재료가 익으면 참기름을 두르고 살짝 더 볶아서 마무리한다.
7. 그릇에 담아 깨를 뿌려 완성한다.

**걸리는 시간:** 30분

**난이도:** 중급